In [167]:
import pandas as pd 
import numpy as np 
import math 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as sklmetrics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.copy_on_write = True

%matplotlib inline

# MEN'S ANALYSIS

## Reading in the data

In [168]:
# Men's regular season detailed stats
MRegStats = pd.read_csv("Data/MRegularSeasonDetailedResults.csv")

# Men's regular season compact stats
MCompStats = pd.read_csv("Data/MRegularSeasonCompactResults.csv")

# Men's NCAA tournament detailed stats
MTournStats = pd.read_csv("Data/MNCAATourneyDetailedResults.csv")

# Men's NCAA tournament compact stats
MCompTournStats = pd.read_csv("Data/MNCAATourneyCompactResults.csv")

# Men's Conference tournament stats
MConfTournStats = pd.read_csv("Data/MConferenceTourneyGames.csv")

# Men's Team names
MTeams = pd.read_csv("Data/MTeams.csv")
MTeamSpellings = pd.read_csv("Data/MTeamSpellings.csv", encoding='unicode_escape')

# Men's Massey Ordinals
MOridinals = pd.read_csv("Data/MMasseyOrdinals.csv")

# Men's Tournament Seeds
MSeeds = pd.read_csv("Data/MNCAATourneySeeds.csv")

# Men's Coaches
MCoaches = pd.read_csv("Data/MTeamCoaches.csv")

# Men's Conferences
MConferences = pd.read_csv("Data/MTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
MGameCities = pd.read_csv("Data/MGameCities.csv")

Dean Oliver created the "Four Factors" of basketball: effective Field Goal%, Offensive Rebound %, Turnover %, and Free Throw Rate. Teams that excel at all or most of these end up performing well overall, so we decided to add them into our analysis. They may not turn out to be useful but we added them just in case.

In [169]:
# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
MRegStats["WTempo"] = MRegStats["WFGA"] - MRegStats["WOR"] + MRegStats["WTO"] + 0.475 * MRegStats["WFTA"]
MRegStats["LTempo"] = MRegStats["LFGA"] - MRegStats["LOR"] + MRegStats["LTO"] + 0.475 * MRegStats["LFTA"]

# Four Factors
# eFG%
MRegStats["WeFG"] = (MRegStats["WFGM"] + 0.5 * MRegStats["WFGM3"]) / MRegStats["WFGA"]
MRegStats["LeFG"] = (MRegStats["LFGM"] + 0.5 * MRegStats["LFGM3"]) / MRegStats["LFGA"]

# OR%
MRegStats["WORPerc"] = MRegStats["WOR"] / (MRegStats["WOR"] + MRegStats["LDR"])
MRegStats["LORPerc"] = MRegStats["LOR"] / (MRegStats["LOR"] + MRegStats["WDR"])

# TO%
MRegStats["WTOPerc"] = (MRegStats["WTO"] / MRegStats["WTempo"]) * 100
MRegStats["LTOPerc"] = (MRegStats["LTO"] / MRegStats["LTempo"]) * 100

# FTR
MRegStats["WFTR"] = MRegStats["WFTA"] / MRegStats["WFGA"]
MRegStats["LFTR"] = MRegStats["LFTA"] / MRegStats["LFGA"]

Net rating is one of the main ways we will go about predicting the winning probability. We believe that the best teams are the most efficient, and that really comes down to points per possession subtracted by your opponent's points per possession. There are other ways to adjust it based on how many turnovers versus missed shots (as one is clearly a sign of a worse offense) but for now we will stick with the unadjusted Net Rating.

In [170]:
# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
MRegStats["WuORTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100
MRegStats["LuORTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
MRegStats["WuDRTG"] = (MRegStats["LScore"] / MRegStats["LTempo"]) * 100
MRegStats["LuDRTG"] = (MRegStats["WScore"] / MRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
MRegStats["WuNetRtg"] = MRegStats["WuORTG"] - MRegStats["WuDRTG"]
MRegStats["LuNetRtg"] = MRegStats["LuORTG"] - MRegStats["LuDRTG"]


Adjusting the data to be longer instead of wider. Twice as long but groupable by team now.

In [171]:
def transform_team_stats(df):
    """
    Transforms a DataFrame containing 'W' (Win) and 'L' (Loss) prefixed columns 
    into a format where both team and opponent perspectives are considered.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with 'W' and 'L' columns.

    Returns:
        pd.DataFrame: Transformed DataFrame with both perspectives.
    """
    # Create copies for team and opponent perspectives
    WTeamStats = df.copy()
    LTeamStats = df.copy()

    # Swap home/away location in LTeamStats
    if "WLoc" in LTeamStats.columns:
        LTeamStats["WLoc"] = LTeamStats["WLoc"].map(lambda x: "H" if x == "A" else "A" if x == "H" else x)
        LTeamStats["LLoc"] = LTeamStats["WLoc"]
        LTeamStats.drop("WLoc", axis=1, inplace=True)

    # Rename columns: WTeamStats (Team perspective)
    WTeamStats.rename(columns={col: col.replace("W", "Team_", 1) for col in WTeamStats.columns if col.startswith("W")}, inplace=True)
    WTeamStats.rename(columns={col: col.replace("L", "Opp_", 1) for col in WTeamStats.columns if col.startswith("L")}, inplace=True)

    # Rename columns: LTeamStats (Opponent perspective)
    LTeamStats.rename(columns={col: col.replace("W", "Opp_", 1) for col in LTeamStats.columns if col.startswith("W")}, inplace=True)
    LTeamStats.rename(columns={col: col.replace("L", "Team_", 1) for col in LTeamStats.columns if col.startswith("L")}, inplace=True)

    # Concatenate both perspectives
    TournTeamStats = pd.concat([WTeamStats, LTeamStats], ignore_index=True)

    return TournTeamStats


Here we transform the regular season and tournament dataframes from a wide, W vs L column structure to a long Team vs Opp structure.

In [172]:
MTeamStats = transform_team_stats(MRegStats)
MTournTeamStats = transform_team_stats(MTournStats)

Adding in conference stats

In [173]:
# Merge conference names for conference bias later in the analysis
MTeamStats = MTeamStats.merge(MConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
MTeamStats = MTeamStats.merge(MConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
MTeamStats["ConfGame"] = (MTeamStats["Team_Conf"] == MTeamStats["Opp_Conf"]).astype(int)

MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Opp_uORTG,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame
0,2003,10,1104,68,1328,62,N,0,27,58,...,86.773968,86.773968,90.006618,3.232650,-3.232650,1104,sec,1328,big_twelve,0
1,2003,10,1272,70,1393,63,N,0,26,62,...,91.970803,91.970803,101.412532,9.441729,-9.441729,1272,cusa,1393,big_east,0
2,2003,11,1266,73,1437,61,N,0,24,58,...,93.954563,93.954563,112.697800,18.743237,-18.743237,1266,cusa,1437,big_east,0
3,2003,11,1296,56,1457,50,N,0,18,38,...,86.021505,86.021505,95.359728,9.338222,-9.338222,1296,mac,1457,big_south,0
4,2003,11,1400,77,1208,71,N,0,30,61,...,111.241676,111.241676,119.984418,8.742741,-8.742741,1400,big_twelve,1208,sec,0


In [174]:
# Add a result to see if a team won or not; this will be used for win percentage later
MTeamStats["Result"] = (MTeamStats["Team_Score"] > MTeamStats["Opp_Score"]).astype(int)

Instead of looking at just net rating, we want to adjust based on the offensive and defensive ratings as well. We normalize the offense and defensive ratings in order to get an adjustment for later. We used a multiplicative model because we wanted there to be more separation between teams both within and between conferences.

In [175]:
# Select only numerical columns in the MTeamStats dataframe
TeamStatsNumericCols = MTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
MTeamGroupedStats = MTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
MConfNetStats = MTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
MConfNetStats["Team_uORTGweight"] = MConfNetStats["Team_uORTG"] / MConfNetStats["Team_uORTG"].mean()
MConfNetStats["Team_uDRTGweight"] = MConfNetStats["Team_uDRTG"] / MConfNetStats["Team_uDRTG"].mean()
#MConfNetStats["Team_uNetweight"] = (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]) / (MConfNetStats["Team_uORTG"] - MConfNetStats["Team_uDRTG"]).mean()

In [176]:
# Merging the conference weights back into the original dataframe
MTeamGroupedStats = MTeamGroupedStats.merge(MConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight"]],on = ["Season","Team_Conf"])

In [177]:
# Creating the conference-adjusted team weights
MTeamGroupedStats["ConfAdjORTG"] = MTeamGroupedStats["Team_uORTG"] * MTeamGroupedStats["Team_uORTGweight"]
MTeamGroupedStats["ConfAdjDRTG"] = MTeamGroupedStats["Team_uDRTG"] * MTeamGroupedStats["Team_uDRTGweight"]
MTeamGroupedStats["ConfAdjustment"] = (MTeamGroupedStats["ConfAdjORTG"] - MTeamGroupedStats["ConfAdjDRTG"])
MTeamGroupedStats["ConfAdjNetRtg"] = MTeamGroupedStats["Team_uNetRtg"] + MTeamGroupedStats["ConfAdjustment"]

In [178]:
MTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjustment"].mean()

Team_TeamID
1101    -6.302463
1102     1.651105
1103     7.152810
1104    13.362988
1105   -17.258504
          ...    
1476   -14.526650
1477   -16.790496
1478   -14.347080
1479   -18.395853
1480   -21.491918
Name: ConfAdjustment, Length: 371, dtype: float64

In [179]:
# Lets add the conference bias onto all of the box scores now
MTeamStats = MTeamStats.merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjustment"]], on = ["Season","Team_TeamID"])

In [180]:
# The new adjusted net rating takes the conference adjustment and adds it to the 
MTeamStats["ConfAdjNetRtg"] = MTeamStats["Team_uNetRtg"] + MTeamStats["ConfAdjustment"]
MTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result,ConfAdjustment,ConfAdjNetRtg
0,2003,10,1104,68,1328,62,N,0,27,58,...,3.232650,-3.232650,1104,sec,1328,big_twelve,0,1,12.510211,15.742861
1,2003,10,1272,70,1393,63,N,0,26,62,...,9.441729,-9.441729,1272,cusa,1393,big_east,0,1,15.290223,24.731952
2,2003,11,1266,73,1437,61,N,0,24,58,...,18.743237,-18.743237,1266,cusa,1437,big_east,0,1,19.075242,37.818479
3,2003,11,1296,56,1457,50,N,0,18,38,...,9.338222,-9.338222,1296,mac,1457,big_south,0,1,-0.418668,8.919554
4,2003,11,1400,77,1208,71,N,0,30,61,...,8.742741,-8.742741,1400,big_twelve,1208,sec,0,1,21.016868,29.759609


The Net Rating is good for regular season predictions, but for the tournament every team is on a much closer scale. Adding a seeding bias will help increase the separation.

In [181]:
MCompSeedStats = (MCompTournStats[MCompTournStats["Season"] > 2002].merge(MSeeds, 
                                                                          left_on = ["Season","WTeamID"], 
                                                                          right_on = ["Season","TeamID"])
                       .drop(["TeamID"], axis = 1)
                       .rename(columns = {"Seed":"WSeed"})
                       .merge(MSeeds, 
                              left_on = ["Season","LTeamID"], 
                              right_on = ["Season","TeamID"])
                       .drop(["TeamID"], axis = 1)
                       .rename(columns = {"Seed":"LSeed"})
                       .merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]],
                              left_on = ["Season","WTeamID"],
                              right_on = ["Season","Team_TeamID"])
                       .drop(["Team_TeamID"], axis = 1)
                       .rename(columns = {"ConfAdjNetRtg": "WNetRtg"})
                       .merge(MTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]],
                              left_on = ["Season","LTeamID"],
                              right_on = ["Season","Team_TeamID"])
                       .drop(["Team_TeamID"], axis = 1)
                       .rename(columns = {"ConfAdjNetRtg": "LNetRtg"}))

In [182]:
def remove_letters(input_string):
  return re.sub(r"[a-zA-Z]", "", input_string)

In [183]:
# Creating wins splits for the seeds both with the region and without the region
MCompSeedStats["WSeedNum"] = [remove_letters(x) for x in MCompSeedStats["WSeed"]]
MCompSeedStats["LSeedNum"] = [remove_letters(x) for x in MCompSeedStats["LSeed"]]
MCompSeedStats["WSeedNum"] = MCompSeedStats["WSeedNum"].astype('int')
MCompSeedStats["LSeedNum"] = MCompSeedStats["LSeedNum"].astype('int')
MCompSeedStats = transform_team_stats(MCompSeedStats)
MCompSeedStats["Result"] = (MCompSeedStats["Team_Score"] > MCompSeedStats["Opp_Score"]).astype('int')
MCompSeedStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_Seed,Opp_Seed,Team_NetRtg,Opp_NetRtg,Team_SeedNum,Opp_SeedNum,Result
0,2003,134,1421,92,1411,84,N,1,X16b,X16a,-25.400217,2.106278,16,16,1
1,2003,136,1112,80,1436,51,N,0,Z01,Z16,41.317106,11.990981,1,16,1
2,2003,136,1113,84,1272,71,N,0,Z10,Z07,24.705251,27.385027,10,7,1
3,2003,136,1141,79,1166,73,N,0,Z11,Z06,16.432594,42.222080,11,6,1
4,2003,136,1143,76,1301,74,N,1,W08,W09,16.714438,19.760664,8,9,1


### Men's Logistic Regression by Seed and Net Rating

In [184]:
MLogRegData = MCompSeedStats[MCompSeedStats["Season"] < 2023].loc[:,["Team_NetRtg","Opp_NetRtg","Team_SeedNum","Opp_SeedNum","Result"]]
#MLogRegData = MCompSeedStats.loc[:,["Team_SeedNum","Opp_SeedNum","Result"]]

MReg_x = MLogRegData.drop(["Result"], axis = 1)
MReg_y = MLogRegData["Result"]

MRegTrain_x,MRegTest_x,MRegTrain_y,MRegTest_y = train_test_split(MReg_x,MReg_y,random_state = 1812,train_size = 0.7)

log_reg_model = LogisticRegression()

log_reg_model.fit(MRegTrain_x,MRegTrain_y)

MPredict_y = log_reg_model.predict(MRegTest_x)

conf_mat = sklmetrics.confusion_matrix(MRegTest_y, MPredict_y, labels =[0,1])
print(sklmetrics.accuracy_score(MRegTest_y, MPredict_y))
print(conf_mat)

0.6969292389853138
[[260 113]
 [114 262]]


In [185]:
# Thank you to W3 schools for this code
def logit2prob(logr, x):
    log_odds = np.dot(logr.coef_, x) + logr.intercept_
    odds = np.exp(log_odds)
    probability = odds / (1 + odds)
    return probability[0] 

In [186]:
MTestCompTourney = MCompTournStats[MCompTournStats["Season"] == 2023]
MTestCompTourney = (MTestCompTourney.merge(MCompSeedStats.loc[:,["Season","DayNum","Team_TeamID","Team_SeedNum","Team_NetRtg"]],
                                           left_on = ["Season","DayNum","WTeamID"],
                                           right_on = ["Season","DayNum","Team_TeamID"])
                                    .rename(columns = {"Team_SeedNum": "WSeedNum",
                                                       "Team_NetRtg": "WNetRtg"})
                                    .drop(["Team_TeamID"], axis = 1)
                                    .merge(MCompSeedStats.loc[:,["Season","DayNum","Team_TeamID","Team_SeedNum","Team_NetRtg"]],
                                           left_on = ["Season","DayNum","LTeamID"],
                                           right_on = ["Season","DayNum","Team_TeamID"])
                                    .rename(columns = {"Team_SeedNum": "LSeedNum",
                                                       "Team_NetRtg": "LNetRtg"})
                                    .drop(["Team_TeamID"], axis = 1))

MTestCompTourney["logitPerc"] = logit2prob(log_reg_model,[MTestCompTourney["WNetRtg"],MTestCompTourney["LNetRtg"],MTestCompTourney["WSeedNum"],MTestCompTourney["LSeedNum"]])
#MTestCompTourney["logitPerc"] = logit2prob(log_reg_model,[MTestCompTourney["WSeedNum"],MTestCompTourney["LSeedNum"]])
MTestCompTourney["logitBrier"] = (MTestCompTourney["logitPerc"] - 1)**2
MTestCompTourney["logitBrier"].mean()

0.20594530622288582

# WOMEN'S ANALYSIS

In [187]:
# Women's regular season detailed stats
WRegStats = pd.read_csv("Data/WRegularSeasonDetailedResults.csv")

# Women regular season compact stats
WCompStats = pd.read_csv("Data/WRegularSeasonCompactResults.csv")

# Women's NCAA tournament detailed stats
WTournStats = pd.read_csv("Data/WNCAATourneyDetailedResults.csv")

# Women's NCAA tournament compact stats
WCompTournStats = pd.read_csv("Data/WNCAATourneyCompactResults.csv")

# Women's Conference tournament stats
WConfTournStats = pd.read_csv("Data/WConferenceTourneyGames.csv")

# Women's Team names
WTeams = pd.read_csv("Data/WTeams.csv")
WTeamSpellings = pd.read_csv("Data/WTeamSpellings.csv", encoding='unicode_escape')

# Women's Tournament Seeds
WSeeds = pd.read_csv("Data/WNCAATourneySeeds.csv")

# Women's Conferences
WConferences = pd.read_csv("Data/WTeamConferences.csv")

# Cities
Cities = pd.read_csv("Data/Cities.csv")
WGameCities = pd.read_csv("Data/WGameCities.csv")

In [188]:
# Tempo
# This is how the NET rankings calculate tempo so we will emulate it
WRegStats["WTempo"] = WRegStats["WFGA"] - WRegStats["WOR"] + WRegStats["WTO"] + 0.475 * WRegStats["WFTA"]
WRegStats["LTempo"] = WRegStats["LFGA"] - WRegStats["LOR"] + WRegStats["LTO"] + 0.475 * WRegStats["LFTA"]

# Net Rating Stats

# "uORTG" is unadjusted Offensive Rating. This will be used to adjust later.
WRegStats["WuORTG"] = (WRegStats["WScore"] / WRegStats["WTempo"]) * 100
WRegStats["LuORTG"] = (WRegStats["LScore"] / WRegStats["LTempo"]) * 100

# The same goes for defensive rating. "uDRTG" is the unadjusted defensive rating
WRegStats["WuDRTG"] = (WRegStats["LScore"] / WRegStats["LTempo"]) * 100
WRegStats["LuDRTG"] = (WRegStats["WScore"] / WRegStats["WTempo"]) * 100

# And now Net Rating is just Offensive Rating - Defensive Rating
WRegStats["WuNetRtg"] = WRegStats["WuORTG"] - WRegStats["WuDRTG"]
WRegStats["LuNetRtg"] = WRegStats["LuORTG"] - WRegStats["LuDRTG"]

In [189]:
# Transform dataframes from wide to long format
WTeamStats = transform_team_stats(WRegStats)
WTournTeamStats = transform_team_stats(WTournStats)

### Adding Conferences to WBB Stats

In [190]:
# Adding Conferences
WTeamStats = WTeamStats.merge(WConferences,left_on = ["Season","Team_TeamID"],right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Team_Conf"})
WTeamStats = WTeamStats.merge(WConferences, left_on = ["Season","Opp_TeamID"], right_on = ["Season","TeamID"]).rename(columns = {"ConfAbbrev": "Opp_Conf"})
WTeamStats["ConfGame"] = (WTeamStats["Team_Conf"] == WTeamStats["Opp_Conf"]).astype(int)

# Add a result to see if a team won or not; this will be used for win percentage later
WTeamStats["Result"] = (WTeamStats["Team_Score"] > WTeamStats["Opp_Score"]).astype(int)

WTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uDRTG,Opp_uDRTG,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result
0,2010,11,3103,63,3237,49,H,0,23,54,...,69.257951,88.701162,19.443211,-19.443211,3103,mac,3237,summit,0,1
1,2010,11,3104,73,3399,68,N,0,26,62,...,83.104186,92.055485,8.951300,-8.951300,3104,sec,3399,ovc,0,1
2,2010,11,3110,71,3224,59,A,0,29,62,...,88.158386,106.446777,18.288390,-18.288390,3110,patriot,3224,meac,0,1
3,2010,11,3111,63,3267,58,A,0,27,52,...,77.462437,81.527014,4.064577,-4.064577,3111,southern,3267,cusa,0,1
4,2010,11,3119,74,3447,70,H,1,30,74,...,90.938616,97.081010,6.142394,-6.142394,3119,patriot,3447,nec,0,1


In [191]:
# Select only numerical columns in the WTeamStats dataframe
TeamStatsNumericCols = WTeamStats.select_dtypes(include = ['number']).columns.difference(["Season","Team_TeamID","Opp_TeamID"])

# Use the numerical columns to then get the mean of all of the columns
WTeamGroupedStats = WTeamStats.groupby(["Season","Team_TeamID","Team_Conf"])[TeamStatsNumericCols].mean().reset_index()

# Group by conference to get conference adjustments
WConfNetStats = WTeamStats.groupby(["Season","Team_Conf"])[["Team_uORTG","Team_uDRTG","Team_uNetRtg"]].mean().reset_index()
WConfNetStats["Team_uORTGweight"] = WConfNetStats["Team_uORTG"] / WConfNetStats["Team_uORTG"].mean()
WConfNetStats["Team_uDRTGweight"] = WConfNetStats["Team_uDRTG"] / WConfNetStats["Team_uDRTG"].mean()
WConfNetStats["Team_uNetweight"] = (WConfNetStats["Team_uORTG"] - WConfNetStats["Team_uDRTG"]) / (WConfNetStats["Team_uORTG"] - WConfNetStats["Team_uDRTG"]).mean()
WTeamGroupedStats = WTeamGroupedStats.merge(WConfNetStats[["Season","Team_Conf","Team_uORTGweight","Team_uDRTGweight","Team_uNetweight"]],on = ["Season","Team_Conf"])

# Creating the conference-adjusted team weights
WTeamGroupedStats["ConfAdjORTG"] = WTeamGroupedStats["Team_uORTG"] * WTeamGroupedStats["Team_uORTGweight"]
WTeamGroupedStats["ConfAdjDRTG"] = WTeamGroupedStats["Team_uDRTG"] * WTeamGroupedStats["Team_uDRTGweight"]
WTeamGroupedStats["ConfAdjustment"] = (WTeamGroupedStats["ConfAdjORTG"] - WTeamGroupedStats["ConfAdjDRTG"]) #* WTeamGroupedStats["Team_uNetweight"]
WTeamGroupedStats["ConfAdjNetRtg"] = WTeamGroupedStats["Team_uNetRtg"] + WTeamGroupedStats["ConfAdjustment"]

In [192]:
WTeamGroupedStats.groupby(["Team_TeamID"])["ConfAdjustment"].mean()

Team_TeamID
3101     0.831935
3102   -14.865361
3103     0.713604
3104    14.193826
3105   -13.762719
          ...    
3476   -22.065787
3477   -13.735953
3478   -26.394570
3479   -22.032496
3480   -10.484587
Name: ConfAdjustment, Length: 366, dtype: float64

In [193]:
# Lets add the conference bias onto all of the box scores now
WTeamStats = WTeamStats.merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjustment"]], on = ["Season","Team_TeamID"])

WTeamStats["ConfAdjNetRtg"] = WTeamStats["Team_uNetRtg"] + WTeamStats["ConfAdjustment"]
WTeamStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_FGM,Team_FGA,...,Team_uNetRtg,Opp_uNetRtg,TeamID_x,Team_Conf,TeamID_y,Opp_Conf,ConfGame,Result,ConfAdjustment,ConfAdjNetRtg
0,2010,11,3103,63,3237,49,H,0,23,54,...,19.443211,-19.443211,3103,mac,3237,summit,0,1,4.902578,24.345789
1,2010,11,3104,73,3399,68,N,0,26,62,...,8.951300,-8.951300,3104,sec,3399,ovc,0,1,4.885961,13.837261
2,2010,11,3110,71,3224,59,A,0,29,62,...,18.288390,-18.288390,3110,patriot,3224,meac,0,1,6.998174,25.286564
3,2010,11,3111,63,3267,58,A,0,27,52,...,4.064577,-4.064577,3111,southern,3267,cusa,0,1,1.466903,5.531480
4,2010,11,3119,74,3447,70,H,1,30,74,...,6.142394,-6.142394,3119,patriot,3447,nec,0,1,-10.717682,-4.575289


### Logistic Regression for WBB Stats

Here we create the dataframe that is only the team and opponent net rankings along with the result.

This is slightly worse than without the logistic regression. One of the games that's hurting us the most is NC State vs Texas in the Elite 8. NC State was a 3 seed and Texas was a 1 seed. In most sites that we've found, Texas had about a 75% chance to win, but in all of our analyses they've had well over 85% chance.  This is because of the massive difference in net rating between the two. The seeds are really telling the story here, however. A 3 seed defeating a 1 seed is an upset, but not nearly as much as a team winning with a difference in net rating of over 30. We need to adjust for seeding.

In [194]:
WCompSeedStats = (WCompTournStats[WCompTournStats["Season"] > 2009].merge(WSeeds, 
                                                                          left_on = ["Season","WTeamID"], 
                                                                          right_on = ["Season","TeamID"])
                       .drop(["TeamID"], axis = 1)
                       .rename(columns = {"Seed":"WSeed"})
                       .merge(WSeeds, 
                              left_on = ["Season","LTeamID"], 
                              right_on = ["Season","TeamID"])
                       .drop(["TeamID"], axis = 1)
                       .rename(columns = {"Seed":"LSeed"})
                       .merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]],
                              left_on = ["Season","WTeamID"],
                              right_on = ["Season","Team_TeamID"])
                       .drop(["Team_TeamID"], axis = 1)
                       .rename(columns = {"ConfAdjNetRtg": "WNetRtg"})
                       .merge(WTeamGroupedStats.loc[:,["Season","Team_TeamID","ConfAdjNetRtg"]],
                              left_on = ["Season","LTeamID"],
                              right_on = ["Season","Team_TeamID"])
                       .drop(["Team_TeamID"], axis = 1)
                       .rename(columns = {"ConfAdjNetRtg": "LNetRtg"}))

WCompSeedStats["WSeedNum"] = [remove_letters(x) for x in WCompSeedStats["WSeed"]]
WCompSeedStats["LSeedNum"] = [remove_letters(x) for x in WCompSeedStats["LSeed"]]
WCompSeedStats["WSeedNum"] = WCompSeedStats["WSeedNum"].astype('int')
WCompSeedStats["LSeedNum"] = WCompSeedStats["LSeedNum"].astype('int')

WCompSeedStats = transform_team_stats(WCompSeedStats)
WCompSeedStats["Result"] = (WCompSeedStats["Team_Score"] > WCompSeedStats["Opp_Score"]).astype('int')
WCompSeedStats.head()

,Season,DayNum,Team_TeamID,Team_Score,Opp_TeamID,Opp_Score,Team_Loc,NumOT,Team_Seed,Opp_Seed,Team_NetRtg,Opp_NetRtg,Team_SeedNum,Opp_SeedNum,Result
0,2010,138,3124,69,3201,55,N,0,X04,X13,49.902714,36.678223,4,13,1
1,2010,138,3173,67,3395,66,N,0,X08,X09,32.830150,35.150654,8,9,1
2,2010,138,3181,72,3214,37,H,0,X02,X15,50.172532,19.521519,2,15,1
3,2010,138,3199,75,3256,61,H,0,W03,W14,45.877331,28.310571,3,14,1
4,2010,138,3207,62,3265,42,N,0,X05,X12,35.892601,29.286494,5,12,1


#### Tourney Seeds Logistic Regression

In [195]:
WLogRegData = WCompSeedStats[WCompSeedStats["Season"] < 2023].loc[:,["Team_NetRtg","Opp_NetRtg","Team_SeedNum","Opp_SeedNum","Team_Loc","Result"]]
WLogRegData["Team_Home"] = (WLogRegData["Team_Loc"] == "H").astype('int')
WLogRegData["Team_Away"] = (WLogRegData["Team_Loc"] == "A").astype('int')
WLogRegData["Team_Neutral"] = (WLogRegData["Team_Loc"] == "N").astype('int')
WLogRegData.drop(["Team_Loc"],axis = 1,inplace = True)
#WLogRegData = WRollStats.loc[:,["FNetDiff","Result"]]


WReg_x = WLogRegData.drop(["Result"], axis = 1)
WReg_y = WLogRegData["Result"]

WRegTrain_x,WRegTest_x,WRegTrain_y,WRegTest_y = train_test_split(WReg_x,WReg_y,random_state = 1812,train_size = 0.7)

log_reg_model = LogisticRegression()

log_reg_model.fit(WRegTrain_x,WRegTrain_y)

WPredict_y = log_reg_model.predict(WRegTest_x)

conf_mat = sklmetrics.confusion_matrix(WRegTest_y, WPredict_y, labels =[0,1])
print(sklmetrics.accuracy_score(WRegTest_y, WPredict_y))
print(conf_mat)

0.8048245614035088
[[189  51]
 [ 38 178]]


In [196]:
WTestCompTourney = WCompTournStats[WCompTournStats["Season"] == 2023]
WTestCompTourney["Team_Home"] = (WTestCompTourney["WLoc"] == "H").astype('int')
WTestCompTourney["Team_Away"] = (WTestCompTourney["WLoc"] == "A").astype('int')
WTestCompTourney["Team_Neutral"] = (WTestCompTourney["WLoc"] == "N").astype('int')
WTestCompTourney.drop(["WLoc"], axis = 1, inplace = True)
WTestCompTourney = (WTestCompTourney.merge(WCompSeedStats.loc[:,["Season","DayNum","Team_TeamID","Team_SeedNum","Team_NetRtg"]],
                                           left_on = ["Season","DayNum","WTeamID"],
                                           right_on = ["Season","DayNum","Team_TeamID"])
                                    .rename(columns = {"Team_SeedNum": "WSeedNum",
                                                       "Team_NetRtg": "WNetRtg"})
                                    .drop(["Team_TeamID"], axis = 1)
                                    .merge(WCompSeedStats.loc[:,["Season","DayNum","Team_TeamID","Team_SeedNum","Team_NetRtg"]],
                                           left_on = ["Season","DayNum","LTeamID"],
                                           right_on = ["Season","DayNum","Team_TeamID"])
                                    .rename(columns = {"Team_SeedNum": "LSeedNum",
                                                       "Team_NetRtg": "LNetRtg"})
                                    .drop(["Team_TeamID"], axis = 1))

WTestCompTourney["logitPerc"] = logit2prob(log_reg_model,[WTestCompTourney["WNetRtg"],WTestCompTourney["LNetRtg"],WTestCompTourney["WSeedNum"],WTestCompTourney["LSeedNum"],WTestCompTourney["Team_Home"],WTestCompTourney["Team_Away"],WTestCompTourney["Team_Neutral"]])
WTestCompTourney["logitBrier"] = (WTestCompTourney["logitPerc"] - 1)**2
WTestCompTourney["logitBrier"].mean()

0.1497975242719741

# Models for 2025